# Supply Chain Analytics & Machine Learning

This notebook provides comprehensive analysis of supply chain datasets including:
- **Demand Forecasting** using ML models
- **Supplier Performance** optimization
- **Inventory Management** analysis
- **Logistics Efficiency** evaluation
- **Interactive Visualizations**

## Datasets Available:
1. **Orders Data** - Transaction records with costs and suppliers
2. **Supplier Performance** - Quality ratings and risk assessments  
3. **Demand Forecasting** - Time series demand data
4. **Logistics** - Transportation and delivery performance
5. **Inventory** - Stock levels and ABC/XYZ classification


In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("📊 Ready for supply chain analysis!")


✅ Libraries imported successfully!
📊 Ready for supply chain analysis!


In [6]:
# Load all supply chain datasets
print("📂 Loading Supply Chain Datasets...")

# Load datasets
orders = pd.read_csv('datasets/supply_chain_sample.csv')
suppliers = pd.read_csv('datasets/supplier_performance.csv')
demand = pd.read_csv('datasets/demand_forecasting.csv')
logistics = pd.read_csv('datasets/logistics_transport.csv')
inventory = pd.read_csv('datasets/inventory_management.csv')

# Display dataset information
datasets = {
    'Orders': orders,
    'Suppliers': suppliers, 
    'Demand': demand,
    'Logistics': logistics,
    'Inventory': inventory
}

print("\n📊 Dataset Overview:")
print("=" * 50)
for name, df in datasets.items():
    print(f"{name:12} | Shape: {df.shape[0]:4}x{df.shape[1]:2} | Missing: {df.isnull().sum().sum():3}")
    
print(f"\n✅ All datasets loaded successfully!")
print(f"📈 Total records across all datasets: {sum(len(df) for df in datasets.values())}")


📂 Loading Supply Chain Datasets...

📊 Dataset Overview:
Orders       | Shape:   20x22 | Missing:   0
Suppliers    | Shape:   10x15 | Missing:   0
Demand       | Shape:   25x17 | Missing:   0
Logistics    | Shape:   20x24 | Missing:   0
Inventory    | Shape:   19x21 | Missing:   0

✅ All datasets loaded successfully!
📈 Total records across all datasets: 94


## 🔍 Exploratory Data Analysis (EDA)


In [7]:
# Comprehensive EDA
print("🔍 EXPLORATORY DATA ANALYSIS")
print("=" * 50)

# 1. Orders Analysis
print("\n💰 ORDERS ANALYSIS:")
print(f"Total order value: ${orders['total_cost'].sum():,.2f}")
print(f"Average order value: ${orders['total_cost'].mean():,.2f}")
print(f"Orders by category:")
print(orders['category'].value_counts())

# 2. Supplier Analysis  
print("\n⭐ SUPPLIER ANALYSIS:")
print(f"Average quality rating: {suppliers['quality_rating'].mean():.2f}")
print(f"On-time delivery rate: {suppliers['on_time_delivery_rate'].mean():.1%}")
print(f"Risk distribution:")
print(suppliers['risk_level'].value_counts())

# 3. Demand Analysis
print("\n📈 DEMAND ANALYSIS:")
print(f"Average actual demand: {demand['actual_demand'].mean():.2f}")
print(f"Forecast accuracy: {1 - abs(demand['actual_demand'] - demand['forecasted_demand']).mean() / demand['actual_demand'].mean():.1%}")

# 4. Logistics Analysis
print("\n🚚 LOGISTICS ANALYSIS:")
print(f"On-time delivery rate: {len(logistics[logistics['delay_days'] <= 0]) / len(logistics):.1%}")
print(f"Average customer satisfaction: {logistics['customer_satisfaction'].mean():.2f}/5")
print(f"Transport modes:")
print(logistics['transport_mode'].value_counts())

# 5. Inventory Analysis
print("\n📦 INVENTORY ANALYSIS:")
print(f"ABC Classification:")
print(inventory['abc_classification'].value_counts())
print(f"XYZ Classification:")
print(inventory['xyz_classification'].value_counts())


🔍 EXPLORATORY DATA ANALYSIS

💰 ORDERS ANALYSIS:
Total order value: $129,835.00
Average order value: $6,491.75
Orders by category:
category
Electronics    11
Furniture       9
Name: count, dtype: int64

⭐ SUPPLIER ANALYSIS:
Average quality rating: 4.39
On-time delivery rate: 90.7%
Risk distribution:
risk_level
Low       6
Medium    2
High      2
Name: count, dtype: int64

📈 DEMAND ANALYSIS:
Average actual demand: 45.72
Forecast accuracy: 93.3%

🚚 LOGISTICS ANALYSIS:
On-time delivery rate: 45.0%
Average customer satisfaction: 4.42/5
Transport modes:
transport_mode
Ground    11
Air        9
Name: count, dtype: int64

📦 INVENTORY ANALYSIS:
ABC Classification:
abc_classification
C    8
B    7
A    4
Name: count, dtype: int64
XYZ Classification:
xyz_classification
Y    8
Z    6
X    5
Name: count, dtype: int64


## 🤖 Machine Learning: Demand Forecasting


In [8]:
# Demand Forecasting ML Models
print("🤖 DEMAND FORECASTING ANALYSIS")
print("=" * 50)

# Prepare features for ML
feature_columns = ['price', 'competitor_price', 'marketing_spend', 'economic_index', 
                  'weather_factor', 'trend_factor', 'seasonality_factor']

X = demand[feature_columns]
y = demand['actual_demand']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train multiple models
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression()
}

print("Training demand forecasting models...")
results = {}

for name, model in models.items():
    # Train model
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Evaluate
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {'MAE': mae, 'MSE': mse, 'R2': r2}
    
    print(f"\n{name} Performance:")
    print(f"  MAE: {mae:.2f}")
    print(f"  MSE: {mse:.2f}")
    print(f"  R²: {r2:.3f}")

# Feature importance
rf_model = models['Random Forest']
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🎯 Feature Importance (Random Forest):")
print(feature_importance)


🤖 DEMAND FORECASTING ANALYSIS
Training demand forecasting models...

Random Forest Performance:
  MAE: 2.27
  MSE: 6.87
  R²: 0.867

Gradient Boosting Performance:
  MAE: 2.33
  MSE: 5.79
  R²: 0.888

Linear Regression Performance:
  MAE: 6.94
  MSE: 110.85
  R²: -1.142

🎯 Feature Importance (Random Forest):
              feature  importance
0               price    0.218106
5        trend_factor    0.211011
6  seasonality_factor    0.191339
2     marketing_spend    0.186043
1    competitor_price    0.181323
3      economic_index    0.006818
4      weather_factor    0.005359


## 📊 Interactive Visualizations


In [9]:
# Create comprehensive visualizations
print("📊 Creating Interactive Visualizations...")

# 1. Cost Distribution
fig1 = px.histogram(orders, x='total_cost', nbins=20, 
                   title='Order Cost Distribution',
                   labels={'total_cost': 'Total Cost ($)', 'count': 'Frequency'})
fig1.show()

# 2. Supplier Performance Scatter
fig2 = px.scatter(suppliers, x='cost_competitiveness', y='quality_rating',
                 color='risk_level', size='on_time_delivery_rate',
                 hover_data=['supplier_name', 'country'],
                 title='Supplier Performance: Cost vs Quality',
                 labels={'cost_competitiveness': 'Cost Competitiveness',
                        'quality_rating': 'Quality Rating'})
fig2.show()

# 3. Demand Forecast Accuracy
fig3 = px.scatter(demand, x='forecasted_demand', y='actual_demand',
                 title='Demand Forecast Accuracy',
                 labels={'forecasted_demand': 'Forecasted Demand',
                        'actual_demand': 'Actual Demand'})
# Add perfect prediction line
fig3.add_shape(type="line", x0=0, y0=0, x1=demand['actual_demand'].max(), 
               y1=demand['actual_demand'].max(), line=dict(dash="dash", color="red"))
fig3.show()

# 4. Inventory ABC Analysis
fig4 = px.pie(inventory, names='abc_classification', 
             title='ABC Classification Distribution')
fig4.show()

# 5. Logistics Performance by Transport Mode
fig5 = px.box(logistics, x='transport_mode', y='delay_days',
             title='Delivery Delays by Transport Mode',
             labels={'transport_mode': 'Transport Mode', 'delay_days': 'Delay (Days)'})
fig5.show()

print("✅ Interactive visualizations created!")


📊 Creating Interactive Visualizations...


✅ Interactive visualizations created!


## 🎯 Supplier Optimization Analysis


In [10]:
# Supplier Optimization Analysis
print("🎯 SUPPLIER OPTIMIZATION ANALYSIS")
print("=" * 50)

# Calculate composite supplier score
suppliers['composite_score'] = (
    suppliers['on_time_delivery_rate'] * 0.3 +
    suppliers['quality_rating'] * 0.25 +
    suppliers['cost_competitiveness'] * 0.2 +
    suppliers['reliability_score'] * 0.15 +
    suppliers['sustainability_rating'] * 0.1
)

# Rank suppliers
suppliers_ranked = suppliers.sort_values('composite_score', ascending=False)

print("🏆 TOP SUPPLIERS:")
print("-" * 30)
for i, (_, row) in enumerate(suppliers_ranked.head(5).iterrows(), 1):
    print(f"{i}. {row['supplier_name']:20} | Score: {row['composite_score']:.3f} | Risk: {row['risk_level']}")

# Risk analysis
print(f"\n⚠️ RISK ANALYSIS:")
high_risk = suppliers[suppliers['risk_level'] == 'High']
print(f"High-risk suppliers: {len(high_risk)}")
if len(high_risk) > 0:
    print("High-risk suppliers:", high_risk['supplier_name'].tolist())

# Cost vs Quality analysis by country
print(f"\n💰 COST VS QUALITY BY COUNTRY:")
cost_quality = suppliers.groupby('country').agg({
    'cost_competitiveness': 'mean',
    'quality_rating': 'mean',
    'on_time_delivery_rate': 'mean'
}).round(3)
print(cost_quality)

# Supplier clustering for optimization
print(f"\n🔍 SUPPLIER CLUSTERING:")
# Prepare features for clustering
cluster_features = ['on_time_delivery_rate', 'quality_rating', 'cost_competitiveness', 
                   'reliability_score', 'sustainability_rating']
X_cluster = suppliers[cluster_features]

# K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
suppliers['cluster'] = kmeans.fit_predict(X_cluster)

cluster_summary = suppliers.groupby('cluster').agg({
    'supplier_name': 'count',
    'on_time_delivery_rate': 'mean',
    'quality_rating': 'mean',
    'cost_competitiveness': 'mean'
}).round(3)

print("Supplier Clusters:")
print(cluster_summary)


🎯 SUPPLIER OPTIMIZATION ANALYSIS
🏆 TOP SUPPLIERS:
------------------------------
1. LogiFlow             | Score: 2.179 | Risk: Low
2. AudioTech            | Score: 2.159 | Risk: Low
3. TechCorp Inc         | Score: 2.138 | Risk: Low
4. MobileTech           | Score: 2.138 | Risk: High
5. DisplayTech          | Score: 2.132 | Risk: Low

⚠️ RISK ANALYSIS:
High-risk suppliers: 2
High-risk suppliers: ['MobileTech', 'GlobalParts']

💰 COST VS QUALITY BY COUNTRY:
             cost_competitiveness  quality_rating  on_time_delivery_rate
country                                                                 
China                       0.935           4.500                  0.885
Germany                     0.780           4.000                  0.920
India                       0.900           4.000                  0.850
Japan                       0.790           4.600                  0.940
Netherlands                 0.750           4.100                  0.910
South Korea                 

## 📦 Inventory Optimization Analysis


In [11]:
# Inventory Optimization Analysis
print("📦 INVENTORY OPTIMIZATION ANALYSIS")
print("=" * 50)

# ABC Analysis
print("📊 ABC ANALYSIS:")
abc_counts = inventory['abc_classification'].value_counts()
print(abc_counts)

# XYZ Analysis  
print(f"\n📈 XYZ ANALYSIS:")
xyz_counts = inventory['xyz_classification'].value_counts()
print(xyz_counts)

# Combined ABC-XYZ Matrix
print(f"\n🎯 ABC-XYZ MATRIX:")
abc_xyz = pd.crosstab(inventory['abc_classification'], inventory['xyz_classification'])
print(abc_xyz)

# Stock optimization recommendations
print(f"\n💡 STOCK OPTIMIZATION RECOMMENDATIONS:")

# Overstocked items
overstocked = inventory[inventory['current_stock'] > inventory['max_stock_level']]
if len(overstocked) > 0:
    print(f"⚠️ Overstocked items: {len(overstocked)}")
    print(overstocked[['product_name', 'current_stock', 'max_stock_level', 'abc_classification']].head())
else:
    print("✅ No overstocked items found")

# Understocked items
understocked = inventory[inventory['current_stock'] < inventory['min_stock_level']]
if len(understocked) > 0:
    print(f"\n📉 Understocked items: {len(understocked)}")
    print(understocked[['product_name', 'current_stock', 'min_stock_level', 'abc_classification']].head())
else:
    print("✅ No understocked items found")

# High turnover items
high_turnover = inventory[inventory['stock_turnover_rate'] > inventory['stock_turnover_rate'].quantile(0.75)]
print(f"\n🚀 High turnover items: {len(high_turnover)}")
print(high_turnover[['product_name', 'stock_turnover_rate', 'abc_classification']].head())

# Inventory value analysis
inventory['inventory_value'] = inventory['current_stock'] * inventory['holding_cost_per_unit']
total_inventory_value = inventory['inventory_value'].sum()
print(f"\n💰 TOTAL INVENTORY VALUE: ${total_inventory_value:,.2f}")

# ABC value analysis
abc_value = inventory.groupby('abc_classification')['inventory_value'].sum().sort_values(ascending=False)
print(f"\n📊 INVENTORY VALUE BY ABC CLASSIFICATION:")
for classification, value in abc_value.items():
    percentage = (value / total_inventory_value) * 100
    print(f"{classification}: ${value:,.2f} ({percentage:.1f}%)")


📦 INVENTORY OPTIMIZATION ANALYSIS
📊 ABC ANALYSIS:
abc_classification
C    8
B    7
A    4
Name: count, dtype: int64

📈 XYZ ANALYSIS:
xyz_classification
Y    8
Z    6
X    5
Name: count, dtype: int64

🎯 ABC-XYZ MATRIX:
xyz_classification  X  Y  Z
abc_classification         
A                   2  2  0
B                   3  4  0
C                   0  2  6

💡 STOCK OPTIMIZATION RECOMMENDATIONS:
✅ No overstocked items found
✅ No understocked items found

🚀 High turnover items: 5
     product_name  stock_turnover_rate abc_classification
1  Wireless Mouse                 12.3                  B
3        Keyboard                 10.2                  B
4          Webcam                  9.6                  B
5      Headphones                 11.4                  B
8  External Drive                  8.8                  B

💰 TOTAL INVENTORY VALUE: $5,852.00

📊 INVENTORY VALUE BY ABC CLASSIFICATION:
A: $4,010.00 (68.5%)
C: $981.00 (16.8%)
B: $861.00 (14.7%)


## 🚚 Logistics Efficiency Analysis


In [12]:
# Logistics Efficiency Analysis
print("🚚 LOGISTICS EFFICIENCY ANALYSIS")
print("=" * 50)

# Delivery performance
on_time = logistics[logistics['delay_days'] <= 0]
on_time_rate = len(on_time) / len(logistics)
print(f"📦 DELIVERY PERFORMANCE:")
print(f"On-time deliveries: {len(on_time)}/{len(logistics)} ({on_time_rate:.1%})")
print(f"Average delay: {logistics['delay_days'].mean():.1f} days")

# Cost analysis by transport mode
print(f"\n💰 COST ANALYSIS BY TRANSPORT MODE:")
cost_by_mode = logistics.groupby('transport_mode').agg({
    'total_cost': ['mean', 'sum'],
    'distance_km': 'mean',
    'route_efficiency': 'mean'
}).round(2)
print(cost_by_mode)

# Carrier performance
print(f"\n🚚 CARRIER PERFORMANCE:")
carrier_perf = logistics.groupby('carrier_name').agg({
    'delay_days': 'mean',
    'customer_satisfaction': 'mean',
    'route_efficiency': 'mean',
    'total_cost': 'mean'
}).round(2)
print(carrier_perf.sort_values('customer_satisfaction', ascending=False))

# Weather impact analysis
print(f"\n🌤️ WEATHER IMPACT ANALYSIS:")
weather_impact = logistics.groupby('weather_conditions').agg({
    'delay_days': 'mean',
    'route_efficiency': 'mean',
    'customer_satisfaction': 'mean'
}).round(2)
print(weather_impact)

# Route optimization opportunities
print(f"\n🎯 ROUTE OPTIMIZATION OPPORTUNITIES:")
low_efficiency = logistics[logistics['route_efficiency'] < logistics['route_efficiency'].quantile(0.25)]
print(f"Low efficiency routes: {len(low_efficiency)}")
if len(low_efficiency) > 0:
    print("Low efficiency routes:")
    print(low_efficiency[['origin_warehouse', 'destination_city', 'route_efficiency', 'distance_km']].head())

# Cost per km analysis
logistics['cost_per_km'] = logistics['total_cost'] / logistics['distance_km']
print(f"\n💵 COST PER KM ANALYSIS:")
print(f"Average cost per km: ${logistics['cost_per_km'].mean():.2f}")
print(f"Most expensive routes:")
expensive_routes = logistics.nlargest(3, 'cost_per_km')[['origin_warehouse', 'destination_city', 'cost_per_km', 'transport_mode']]
print(expensive_routes)


🚚 LOGISTICS EFFICIENCY ANALYSIS
📦 DELIVERY PERFORMANCE:
On-time deliveries: 9/20 (45.0%)
Average delay: 0.7 days

💰 COST ANALYSIS BY TRANSPORT MODE:
               total_cost       distance_km route_efficiency
                     mean   sum        mean             mean
transport_mode                                              
Air                307.78  2770        0.00             0.96
Ground             645.91  7105     2127.27             0.85

🚚 CARRIER PERFORMANCE:
              delay_days  customer_satisfaction  route_efficiency  total_cost
carrier_name                                                                 
DHL                -1.00                   4.68              0.97      311.25
FedEx              -0.14                   4.59              0.94      365.00
UPS                 2.00                   4.18              0.84      675.00

🌤️ WEATHER IMPACT ANALYSIS:
                    delay_days  route_efficiency  customer_satisfaction
weather_conditions             

## 📈 Key Insights & Recommendations


In [13]:
# Generate comprehensive insights and recommendations
print("📈 SUPPLY CHAIN INSIGHTS & RECOMMENDATIONS")
print("=" * 60)

insights = []

# Financial insights
total_order_value = orders['total_cost'].sum()
avg_order_value = orders['total_cost'].mean()
insights.append(f"💰 Total order value: ${total_order_value:,.2f}")
insights.append(f"💰 Average order value: ${avg_order_value:,.2f}")

# Supplier insights
high_quality_suppliers = len(suppliers[suppliers['quality_rating'] > 4.5])
high_risk_suppliers = len(suppliers[suppliers['risk_level'] == 'High'])
insights.append(f"⭐ High-quality suppliers (>4.5 rating): {high_quality_suppliers}")
insights.append(f"⚠️ High-risk suppliers: {high_risk_suppliers}")

# Demand forecasting insights
forecast_accuracy = 1 - abs(demand['actual_demand'] - demand['forecasted_demand']).mean() / demand['actual_demand'].mean()
insights.append(f"📈 Forecast accuracy: {forecast_accuracy:.1%}")

# Inventory insights
overstocked_items = len(inventory[inventory['current_stock'] > inventory['max_stock_level']])
understocked_items = len(inventory[inventory['current_stock'] < inventory['min_stock_level']])
insights.append(f"📦 Overstocked items: {overstocked_items}")
insights.append(f"📉 Understocked items: {understocked_items}")

# Logistics insights
on_time_delivery_rate = len(logistics[logistics['delay_days'] <= 0]) / len(logistics)
avg_customer_satisfaction = logistics['customer_satisfaction'].mean()
insights.append(f"🚚 On-time delivery rate: {on_time_delivery_rate:.1%}")
insights.append(f"😊 Average customer satisfaction: {avg_customer_satisfaction:.1f}/5")

print("\n🎯 KEY INSIGHTS:")
for i, insight in enumerate(insights, 1):
    print(f"  {i}. {insight}")

# Recommendations
print(f"\n💡 STRATEGIC RECOMMENDATIONS:")
print("=" * 40)

recommendations = [
    "🎯 SUPPLIER OPTIMIZATION: Focus on high-quality, low-risk suppliers",
    "📊 DEMAND FORECASTING: Improve forecast accuracy with ML models",
    "📦 INVENTORY MANAGEMENT: Implement ABC/XYZ analysis for better stock control",
    "🚚 LOGISTICS: Optimize routes and reduce delivery delays",
    "💰 COST REDUCTION: Negotiate better rates with top-performing carriers",
    "⚠️ RISK MANAGEMENT: Develop contingency plans for high-risk suppliers",
    "📈 PERFORMANCE MONITORING: Implement real-time KPI dashboards",
    "🔄 CONTINUOUS IMPROVEMENT: Regular supplier and process evaluations"
]

for i, rec in enumerate(recommendations, 1):
    print(f"  {i}. {rec}")

print(f"\n✅ Analysis complete! Use these insights to optimize your supply chain operations.")


📈 SUPPLY CHAIN INSIGHTS & RECOMMENDATIONS

🎯 KEY INSIGHTS:
  1. 💰 Total order value: $129,835.00
  2. 💰 Average order value: $6,491.75
  3. ⭐ High-quality suppliers (>4.5 rating): 4
  4. ⚠️ High-risk suppliers: 2
  5. 📈 Forecast accuracy: 93.3%
  6. 📦 Overstocked items: 0
  7. 📉 Understocked items: 0
  8. 🚚 On-time delivery rate: 45.0%
  9. 😊 Average customer satisfaction: 4.4/5

💡 STRATEGIC RECOMMENDATIONS:
  1. 🎯 SUPPLIER OPTIMIZATION: Focus on high-quality, low-risk suppliers
  2. 📊 DEMAND FORECASTING: Improve forecast accuracy with ML models
  3. 📦 INVENTORY MANAGEMENT: Implement ABC/XYZ analysis for better stock control
  4. 🚚 LOGISTICS: Optimize routes and reduce delivery delays
  5. 💰 COST REDUCTION: Negotiate better rates with top-performing carriers
  6. ⚠️ RISK MANAGEMENT: Develop contingency plans for high-risk suppliers
  7. 📈 PERFORMANCE MONITORING: Implement real-time KPI dashboards
  8. 🔄 CONTINUOUS IMPROVEMENT: Regular supplier and process evaluations

✅ Analysis comple